In [ ]:
import pandas as pd
pd.options.plotting.backend = 'plotly'

In [ ]:
vacc_data = pd.read_excel('../data/covid-19-vaccination-vaccination-data-22-december-2022.xlsx', index_col=0)['Value']

In [ ]:
agegroups = ['16-19'] + [f'{age}-{age + 4}' for age in range(20, 95, 5)] + ['95+']
final_vacc_data = pd.DataFrame(
    {
        '1+': vacc_data[[f'Age group - {age} - Number of people who have received at least 1 dose' for age in agegroups]].values,
        '2+': vacc_data[[f'Age group - {age} - Number of people who have received at least 2 doses' for age in agegroups]].values,
        'pop': vacc_data[[f'Age group - {age} - Population' for age in agegroups]].values,
    },
    index=agegroups,
)

In [ ]:
coverage = pd.DataFrame(index=agegroups)
for dose in ['1+', '2+']:
    coverage[dose] = final_vacc_data[dose] / final_vacc_data['pop']
    coverage[dose][coverage[dose] > 1.0] = 1.0

In [ ]:
final_vacc_data.plot()